# Vectors

Ch 1 Interactions and Motion

- §1.4 &ndash; Describing the 3D World: Vectors
- §1.7 &ndash; Predicting a new Position

In [1]:
########
## Env Setup / Preamble
########
%load_ext autoreload
%autoreload complete 

%load_ext Cython

%display latex

################

import cython
from sage.all import *

################

import tempfile
import mmap
from multiprocessing import shared_memory

################

import numpy as np
import scipy.constants

np.random.seed( int(scipy.constants.golden_ratio * 1.e6) )

from numpy import vectorize
from numpy import r_, c_, s_

################

####    boiler-plate class for named constants
from sage.symbolic.constants import Constant
class ConstantFromExpression(Constant):
    def __init__(self, name, v):
        conversions = dict(maxima=repr(maxima.coerce(v)))
        Constant.__init__(self, name, conversions=conversions)
        self._v = v
    def __float__(self): return float(self._v)
    def __complex__(self): return complex(self._v)
    def _mpfr_(self, R): return R(self._v)
    def _real_double_(self, R): return R(self._v)

def NamedExpression(name, expr):
    return ConstantFromExpression(name,expr).expression()
####

import sage.symbolic.units as units     # <: TODO

π   = NamedExpression('π', scipy.constants.pi)          # pi
ε_0 = NamedExpression('ε_0', scipy.constants.epsilon_0) # electric permittivity of vacuum
μ_0 = NamedExpression('μ_0', scipy.constants.mu_0)      # magnetic permeability of vacuum

𝛋 = 1. / (4 * π * ε_0)      # Coulomb's constant in [ N·m²/C² ]
q = NamedExpression('q', -scipy.constants.elementary_charge)

ℎ_planck = NamedExpression('ℎ', scipy.constants.Planck)
ℏ = NamedExpression('ℏ', scipy.constants.hbar)

α = NamedExpression('α', scipy.constants.fine_structure)

m_e = NamedExpression('m_e', scipy.constants.electron_mass)
m_p = NamedExpression('m_p', scipy.constants.proton_mass)

################

Parallelism().set(nproc=8)
print(f"Python: {sys.version}")
version()


Python: 3.12.5 (main, Feb 25 2025, 12:37:41) [Clang 16.0.0 (clang-1600.0.26.6)]


'SageMath version 10.6, Release Date: 2025-03-31'

In [2]:
##  sanity check
import this


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


---

<hr width="82%" style="border-top:2px dashed; border-bottom:none"/>

<hr width="64%" style="border-top:1px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='1' style="display:none">[1]</h2>

>   ---
>   ## [1]
>   
>   __Select each__ of the following vectors that is a __unit vector__:
>
>   - [ ] &emsp; (ⅰ) $\langle\ 1, 2, 3\ \rangle$
>   - [ ] &emsp; (ⅱ) $\langle\ 0, 0, 1\ \rangle$
>   - [ ] &emsp; (ⅲ) $\langle\ 1, 1, 1\ \rangle$
>   - [ ] &emsp; (ⅳ) $\langle\ 1, 0, 0\ \rangle$
>   - [ ] &emsp; (ⅴ) $\langle\ 1/\sqrt{2}, 1/\sqrt{2}, 0\ \rangle$
>   - [ ] &emsp; (ⅵ) $\langle\ 1, 0, 0\ \rangle \left[\mathrm{m/s}\right]$
>   - [ ] &emsp; (ⅵ) $\langle\ 1/\sqrt{3}, 1/\sqrt{3}, 1/\sqrt{3}\ \rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] &emsp; (ⅱ) 
- [x] &emsp; (ⅳ) 
- [x] &emsp; (ⅴ) 
- [x] &emsp; (ⅵ) 
- [x] &emsp; (ⅵ) 

<div style="page-break-after: always;"/> 
<h2 id='2' style="display:none">[2]</h2>

>   ---
>   ## [2]
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='2a' style="display:none">(A)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00b2;&hairsp;&#x2E05;&hairsp;(A)
>
>   __What__ is the __unit vector__ in the direction of the __vector $\vec{A} = \langle\ 54, -31, -67\ \rangle \left[\mathrm{cm}\right]$ ?__
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [7]:
vector(r_[54, -31, -67]).normalized()


(27/4183*sqrt(8366), -31/8366*sqrt(8366), -67/8366*sqrt(8366))

<h3 id='2b' style="display:none">(B)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x00b2;&hairsp;&#x2E05;&hairsp;(B)
>
>   What is the vector $\vec{B}$, in the same direction as $\vec{A} = \langle 54, -31, -67\rangle$ [cm], but with a magnitude of $243$ [cm]?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

In [10]:
243 * vector(r_[54, -31, -67]).normalized()


(6561/4183*sqrt(8366), -7533/8366*sqrt(8366), -16281/8366*sqrt(8366))

<div style="page-break-after: always;"/> 
<h2 id='3' style="display:none">[3]</h2>

>   ---
>   ## [3]
>
>   Select each expression whose result will produce a legal vector: 
>
>   - [ ] &emsp; (ⅰ) $\frac{\langle 7, 1, 3\rangle}{11}$
>   - [ ] &emsp; (ⅱ) $4 \times \langle 6, 3, 3\rangle$
>   - [ ] &emsp; (ⅲ) $7 + \langle\frac{1}{5}, \frac{2}{7}, -4\rangle$
>   - [ ] &emsp; (ⅳ) $\frac{1}{\langle 3, 5, 8\rangle}$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

- [x] &emsp; (ⅰ) $\frac{\langle 7, 1, 3\rangle}{11}$
- [x] &emsp; (ⅱ) $4 \times \langle 6, 3, 3\rangle$


<div style="page-break-after: always;"/> 
<h2 id='4' style="display:none">[4]</h2>

>   ---
>   ## [4]
>
>   A vector can be factored into a product of its magnitude, times the unit vector. 
>
>   Consider the vector $\vec{r} = \langle -10, -28, 23 \rangle$; and factor it 
>   into the form  "number" times "unit vector":
>
>   $$\vec{r} = \left( \fbox{\hspace{2em}?} \right) \left\langle \begin{matrix} \fbox{\hspace{4em}?}, \\ \fbox{\hspace{4em}?}, \\ \fbox{\hspace{4em}?} \end{matrix} \right\rangle$$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='5' style="display:none">[5]</h2>

>   ---
>   ## [5]
>
>   xxx
>   
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [6]
>
>   If $\vec{v} = \langle -2, -7, 2\rangle$, what is the magnitude of $-0.5 \vec{v}$?
>
>   $\fbox{\hspace{4em}?} \left[ \textrm{m/s} \right]$
>   
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='6' style="display:none">[6]</h2>

>   ---
>   ## [7]
>
>   Consider the vectors $\vec{r}_1$ and $\vec{r}_2$, represented by arrows in the 
>   figure. 
>   
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   {{ graphic }}
>
>   Are these two vectors equal?
>
>   - [ ] &nbsp; Yes
>   - [ ] &nbsp; No
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7b' style="display:none">(8)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(B)
>
>   If $\vec{a} = \langle 300, 600, -300 \rangle$ [m/s^2], and $\vec{c} = \vec{a}$,
>   what is the unit vector $\hat{c}$ in the direction of $\vec{c}$?
>
>   $\hat{c} = \left\langle \fbox{\hspace{4em}?}, \fbox{\hspace{4em}?}, \fbox{\hspace{4em}?} \right\rangle$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   A proton traveling with a velocity of $\left\langle 4 \times 10^2, 5 \times 10^2,
>   -5\times 10^2 \right\rangle$ [m/s], passes the origin at a time of $6$ [seconds] 
>   after the proton detector is turned on.  
>
>   Assuming the velocity of the proton does not change, what will its position
>   be, at a time $t = 6.023$ [seconds]?
>
>   $\left\langle \fbox{\hspace{4em}?}, \fbox{\hspace{4em}?}, \fbox{\hspace{4em}?} \right\rangle$ [m]
>   
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='8' style="display:none">[8]</h2>

>   ---
>   ## [8]
>
>   A proton traveling with a velocity of $\left\langle 4 \times 10^2, 5 \times 10^2,
>   -5\times 10^2 \right\rangle$ [m/s], passes the origin at a time of $6$ [seconds] 
>   after the proton detector is turned on.  
>
>   Assuming the velocity of the proton does not change, what will its position
>   be, at a time $t = 6.023$ [seconds]?
>
>   $\left\langle \fbox{\hspace{4em}?}, \fbox{\hspace{4em}?}, \fbox{\hspace{4em}?} \right\rangle$ [m]
>   
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='9' style="display:none">[9]</h2>

>   ---
>   ## [9]
>
>   A comet travels in an elliptical path around a star, in the direction shown
>   in the diagram. 
>
>   {{ graphic }}
>
>   |   |   |   |
>   |:---:|:---:|:---:|
>   | 🔲 h | 🔲 a | 🔲 b |
>   | 🔲 g | &emsp; | 🔲 c |
>   | 🔲 f | 🔲 e | 🔲 d |
>
>   &emsp; 🔲 j - zero magnitude
>   
>   Which arrow best indicates the direction, of the comet's velocity vector, at
>   each of the following locations in the orbit? 
>
>   - location 1
>   - location 2
>   - location 3
>   - location 4
>   - location 5
>   
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   How many numbers are needed to specify a 3D position vector? 
>
>   $\fbox{\hspace{4em}?}$ numbers. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   How many numbers are needed to specify a scalar?
>
>   $\fbox{\hspace{4em}?}$ numbers. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   If $\vec{r} = \left\langle -4, -6, 2 \right\rangle$, find $\|\vec{r}\|$.
>
>   $\fbox{\hspace{4em}?}$ [m]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   Select (checkbox) all of the expressions below that are possible. 
>
>   - [ ] &nbsp; $\|\vec{v}\| > 0$
>
>   - [ ] &nbsp; $\|\vec{v}\| < 0$
>
>   - [ ] &nbsp; $\|\vec{v}\| = 0$
>
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   The vector $\vec{g} = \left\langle 2, -8, 4 \right\rangle$, and the scalar $h = -2$. 
>
>   What is $h + \vec{g}$?
>
>   (select one:)
>
>   - [ ] &nbsp; $\langle 4, -6, 6 \rangle$
>
>   - [ ] &nbsp; $\langle 0, -10, 2 \rangle$
>
>   - [ ] This is a meaningless expression, because a scalar cannot be added to a vector. 
>
>   - [ ] $\langle -4, 16, -8 \rangle$
>
>   - [ ] $\langle -1, 4, -2 \rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   What is the value of $4 / \langle 2, -7, 6 \rangle$? 
>
>   (select one:)
>
>   - [ ] &nbsp; $\langle 8, -28, 24 \rangle$
>
>   - [ ] &nbsp; $\langle 6, -3, 10 \rangle$
>
>   - [ ] $\langle -2, 11, -2 \rangle$
>
>   - [ ] This is a meaningless expression, because a scalar cannot be divide by a vector. 
>
>   - [ ] $\langle 0.5, -1.75, 1.5 \rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   If $\vec{a} = \langle 1, -1, 5 \rangle$, then what is $3\vec{a}$? 
>
>   How does the magnitude of $3\vec{a}$ compare to the magnitude of $\vec{a}$?
>
>   (select one:)
>
>   How does the magnitude of $3\vec{a}$ compare to the magnitude of $\vec{a}$?. 
>
>   - [ ] &nbsp; Not enough information is given.
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is zero. 
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is a factor of $3$ greater than the magnitude of $\vec{a}$. 
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is the same as the magnitude of $\vec{a}$. 
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is a factor of $3$ less than the magnitude of $\vec{a}$. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   If $\vec{a} = \langle 1, -1, 5 \rangle$, then what is $3\vec{a}$? 
>
>   How does the magnitude of $3\vec{a}$ compare to the magnitude of $\vec{a}$?
>
>   (select one:)
>
>   How does the magnitude of $3\vec{a}$ compare to the magnitude of $\vec{a}$?. 
>
>   - [ ] &nbsp; Not enough information is given.
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is zero. 
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is a factor of $3$ greater than the magnitude of $\vec{a}$. 
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is the same as the magnitude of $\vec{a}$. 
>
>   - [ ] &nbsp; The magnitude of $3\vec{a}$ is a factor of $3$ less than the magnitude of $\vec{a}$. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='17' style="display:none">[17]</h2>

>   ---
>   ## [17]
>
>   How does the magnitude of the vector $-\vec{a}$ compare to the magnitude of 
>   the vector $\vec{a}$?
>
>   - [ ] &nbsp; You can't take the magnitude of $-\vec{a}$. 
>
>   - [ ] &nbsp; The magnitude of $-\vec{a}$ is $-1$ times the magnitude of $\vec{a}$.
>
>   - [ ] &nbsp; The two magnitudes are equal. 
>
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='17' style="display:none">[17]</h2>

>   ---
>   ## [18]
>
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the unit vector in the direction of $\vec{r} = \langle 0, 7, 0 \rangle$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the unit vector in the direction of $\vec{r} = \langle 0, 0, -7 \rangle$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='17' style="display:none">[17]</h2>

>   ---
>   ## [18]
>
>   What is the unit vector $\hat{a}$ in the direction of $\vec{a}$, where 
>   $\vec{a} = \langle 400, 700, -300 \rangle$ [m/s^2]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='17' style="display:none">[17]</h2>

>   ---
>   ## [18]
>
>   If $\vec{a} = \langle -3, 3, 7 \rangle$, and $\vec{b} = \vec{a}$, what is the 
>   $y$-component of $\vec{b}$?
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='17' style="display:none">[17]</h2>

>   ---
>   ## [21]
>
>   Consider the vectors:
>
>   - $\vec{F}_1 = \langle 700, 0, -300 \rangle$, and 
>   - $\vec{F}_2 = \langle -400, 0, 200 \rangle$
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of $\vec{F}_1$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of $\vec{F}_2$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of $\left(\vec{F}_1 + \vec{F}_2\right)$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of $\vec{F}_1$, plus the magnitude of $\vec{F}_2$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Is $\|\vec{F}_1 + \vec{F}_2\| = \|\vec{F}_1\| + \|\vec{F}_2\|$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='17' style="display:none">[17]</h2>

>   ---
>   ## [21]
>
>   - $\vec{A}_1 = \langle 800, 600, -800 \rangle$, and 
>   - $\vec{B}_2 = \langle -200, -200, 150 \rangle$
>
>   Calculate the following: 
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\vec{A} + \vec{B}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\|\vec{A} + \vec{B}\|$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\|\vec{A}\|$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\|\vec{B}\|$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\|\vec{A}\| + \|\vec{B}\|$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   A snail is initially at location $\vec{r}_1 = \langle -6, 3, -6 \rangle$.
>
>   At a later time, the snail has crawled to location $\vec{r}_2 = \langle 2, -4, 4 \rangle$ [m].
>
>   What is $\Delta \vec{r}$, the change in the snail's position?
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   Which of the following are vectors? 
>
>   (select all that apply:)
>
>   - [ ] &nbsp; $\langle 0, 2.3, -1 \rangle$
>   - [ ] &nbsp; $3.5$
>   - [ ] &nbsp; $-3 \times 10^{-6}$
>   - [ ] &nbsp; $5 \cdot \langle 0, 2.3, -1 \rangle$
>   - [ ] &nbsp; $0$
>   - [ ] &nbsp; $\langle 0.7, 0.7, -0.7 \rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   If $\vec{p} = \langle -6, 8, 3 \rangle$, what is $6 + \vec{p}$?
>
>   - [ ] &nbsp; This is a meaningless expression, because a scalar cannot be added to a vector. 
>
>   - [ ] &nbsp; $\langle -36, 48, 18 \rangle$
>   - [ ] &nbsp; $\langle -1.00, 1.33, 0.500 \rangle$
>   - [ ] &nbsp; $\langle 0, 14, 9 \rangle$
>   - [ ] &nbsp; $\langle -12, 2, -3 \rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   Which of the following are unit vectors? (Note: Numerical values are given
>   only to 3 significant figures.)
>
>   (select all that apply:)
>
>   - [ ] &nbsp; $\langle 0, 0, -1\rangle$
>   - [ ] &nbsp; $\langle 0.372, -0.557, 0.743 \rangle$
>   - [ ] &nbsp; $\langle 0.949, 0, -0.316 \rangle$
>   - [ ] &nbsp; $\langle 0.9, 0, 0.1 \rangle$
>   - [ ] &nbsp; $\langle 0.5, 0.5, 0 \rangle$
>   - [ ] &nbsp; $\langle 1, 1, 1 \rangle$
>   - [ ] &nbsp; $\langle 0.333, 0.333, 0.333 \rangle$
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   Two vectors, $\vec{r}$ and $\vec{c}$, are equal: $\vec{r} = \vec{c}$. 
>
>   Which of the following statements are true? Select all that apply. 
>
>   - [ ] &nbsp; The $z$-component of $\vec{r}$ must be equal to the $z$-component of $\vec{c}$. 
>   - [ ] &nbsp; The unit vector $\hat{r}$ must be equal to the unit vector $\hat{c}$. 
>   - [ ] &nbsp; The directions of $\vec{r}$ and $\vec{c}$ may be different. 
>   - [ ] &nbsp; The $x$-component of $\vec{r}$ must be equal to the $x$-component of $\vec{c}$.
>   - [ ] &nbsp; The $y$-component of $\vec{r}$ must be equal to the $y$-component of $\vec{c}$. 
>   - [ ] &nbsp; The magnitudes of $\vec{r}$ and $\vec{c}$ may be different. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   Read these questions carefully. They are not the same. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Which of the vectors in the diagram have magnitudes (approximately) equal to
>   the magnitude of $\vec{a}$? 
>
>   (select all that apply:)
>
>   - [ ] &nbsp; $\vec{f}$
>   - [ ] &nbsp; $\vec{d}$
>   - [ ] &nbsp; $\vec{b}$
>   - [ ] &nbsp; $\vec{g}$
>   - [ ] &nbsp; $\vec{e}$
>   - [ ] &nbsp; $\vec{c}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Which of the vectors in the diagram are equal to $\vec{a}$? 
>
>   (select all that apply:)
>
>   - [ ] &nbsp; $\vec{c}$
>   - [ ] &nbsp; $\vec{b}$
>   - [ ] &nbsp; $\vec{e}$
>   - [ ] &nbsp; $\vec{d}$
>   - [ ] &nbsp; $\vec{g}$
>   - [ ] &nbsp; $\vec{f}$
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [23]
>
>   What is the magnitude of the vector $\vec{v}$, where $\vec{v} = 7.7 \times 10^6, 0, -3.6 \times 10^7$ [m/s].
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [29]
>
>   What is the magnitude of the vector $\vec{v}$, where $\vec{v} = 7.7 \times 10^6, 0, -3.6 \times 10^7$ [m/s].
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [29]
>
>   In the diagram below, three vectors are represented by arrows in the $xy$-plane. 
>   
>   Each division represents $1$ meter. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{a}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of the vector $\vec{a}$, $\|\vec{a}\|$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{c}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of the vector $\vec{c}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{g}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of the vector $\vec{g}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   The following questions refer to the vectors depicted by arrows, in the figure below. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{a}$?
>
>   Note that since the vector lies in the $xy$-plane, its $z$-component is zero. 
>
>   Express your answer in vector form. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{b}$?
>
>   (Express your answer in vector form.)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Is this statement `true` or `false`?
>
>   $\vec{a} = \vec{b}$
>
>   - [ ] &nbsp; True
>   - [ ] &nbsp; False
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{c}$?
>
>   (Express your answers in vector form.)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Is this statement true or false?
>
>   $\vec{c} = -\vec{a}$
>
>   - [ ] &nbsp; True
>   - [ ] &nbsp; False
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What are the components of the vector $\vec{d}$?
>
>   (Express your answers in vector form.)
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Is this statement true or false?
>
>   $\vec{d} = -\vec{c}$
>
>   - [ ] &nbsp; True
>   - [ ] &nbsp; False
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [32]
>
>   ...
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   What is the result of multiplying the vector $\vec{a}$ by the scalar $f$,
>   where $\vec{a} = \langle 0.04, -1.6, 20 \rangle$ and $f = 9.0$.
>
>   (Express your answer in vector form.)
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   In the following figure, let $a = 3$ [m], $b = 3$ [m], $c = 2$ [m]. 
>
>   {{ graphic }}
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   In the figure above, what are the components of the vector $\vec{d}$?
>
>   Express your answer in vector form. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   If $\vec{e} = -\vec{d}$, what are the components of $\vec{e}$?
>
>   Express your answer in vector form. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   If the tail of vector $\vec{d}$ were moved to location $\langle -5, -2, 4 \rangle$, where would the tip of the vector be located? 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   If the tail of vector $-\vec{d}$ were placed at location $\langle -1, -1, -1 \rangle$ [m], where would the tip of the vector be located? 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [35]
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the unit vector in the direction of $\langle 2, 2, 2 \rangle$?
>
>   Express your answer in vector form. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the unit vector in the direction of $\langle 4.1, 4.1, 4.1 \rangle$?
>
>   Express your answer in vector form. 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [36]
>
>   Any vector can be written as a unit vector, multiplied by the magnitude of 
>   the vector (a positive scalar). 
>
>   Write each of the following vectors, as the magnitude of the vector times 
>   the appropriate unit vector. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\langle 0, 0, 3.7 \rangle = $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\langle 0, -695, 0 \rangle = $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   $\langle 4\times 10^6, -8 \times 10^6, 5 \times 10^6 \rangle = $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [37]
>
>   A proton is located at $6 \times 10^{-10}, -4 \times 10^{-10}, -7 \times 10^{-10}$ [m].
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is $\vec{r}$, the vector from the origin to the location of the proton? 
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is the magnitude of $\vec{r}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   What is $\hat{r}$, the unit vector (vector with magnitude 1), in the direction 
>   of $\vec{r}$?
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [38]
>
>   In the figure below, the vector $\vec{r}_1$ points to the location of object 1,
>   and $\vec{r}_2$ points to the location of object 2. 
>
>   Both vectors lie in the $xy$-plane. 
>
>   Express your answers in vector form. 
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Calculate the position of object 2 relative to object 1, as a relative position
>   vector. 
>
>   $\vec{r}_{21} = $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<h3 id='7a' style="display:none">(7)</h3>

>   <hr width="82%" style="border-bottom:2px dashed; border-top:none"/>
>
>   ### &#x2e04;&hairsp;&#x2077;&hairsp;&#x2E05;&hairsp;(A)
>
>   Calculate the position of object 1 relative to object 2, as a relative position
>   vector. 
>
>   $\vec{r}_{12} = $
>
>   <hr width="64%" style="border-top:2px dotted; border-bottom:none"/>

<div style="page-break-after: always;"/> 
<h2 id='44' style="display:none">[44]</h2>

>   ---
>   ## [44]
>
>   
>
>   __<p align='right'>(0.1 points)</p>__
>
>   <hr width="82%" style="border-top:1px dashed; border-bottom:none"/>

---